Implementation for blog post

In [ ]:
import math
import json
import pickle
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from transformers import pipeline

# Get candidate labels
with open("packing_label_structure.json", "r") as file:
    candidate_labels = json.load(file)
keys_list = list(candidate_labels.keys())

# Load test data (list of dictionaries)
with open("test_data.json", "r") as file:
    packing_data = json.load(file)
# Extract trip descriptions and classification (trip_types)
trip_descriptions = [trip['description'] for trip in packing_data]
trip_types = [trip['trip_types'] for trip in packing_data]

# First classify using first dimension of classes ()
print(candidate_labels[keys_list[0]])



In [ ]:
classifier = pipeline("zero-shot-classification", model=model_name)
result = classifier(trip_descr, candidate_labels[key])
print(result)